# Création du Dataframe + initialisation de la Data

Pensez a gerer les colonnes en trop excel (df.drop) \
Pour chimiste 2 mso trop long penser a tout mettre dans 1 case \
A faire je pense gestion du profile du gars du type bouton `Mon Profil` avec possibilité de changer année groupe filiere et Mso

In [1]:
import pandas as pd
import json
from numpyencoder import NumpyEncoder

# Read csv file
date = "01_04"
#Df = pd.read_csv(f'./4ETI/{date}.csv', sep=';')
Df  = pd.read_csv('3CGPtest.csv', sep=';')
# Filter columns and rows
Df.reset_index()
Df.fillna(0, inplace=True)
# on ne garde que les 5 premieres columns
Df = Df.iloc[:, 0:5]
Df

,LUNDI,MARDI,MERCREDI,JEUDI,VENDREDI
0,0,0,0,0,0
1,03-avr,04-avr,05-avr,06-avr,07-avr
2,8h00 - 10h00,8h00 - 10h00,8h00 - 10h00,8h00 - 12h15,8h00 - 10h00
3,0,0,0,0,0
4,Chimie Organique,Techniques Chromatographiques,Anglais,T.P.,Techniques Chromatographiques
...,...,...,...,...,...
69,Gr. TP 3A : 13h30 - 15h30,* Les horaires de SHES sont,0,0,0
70,Salle I306,complémentaires à ceux de LV2.,0,0,0
71,Gr. TP 3B : 15h45 - 17h45,Si SHES 13h30-15h30 --> LV2 15h45-17h45,0,0,0
72,Salle I300,Si LV2 13h30-15h30 --> SHES 15h45-17h45,Grand Amphi,0,0


### On récupère la liste de toutes les matière/prof/majeurs dans LstSemaine séparé en Matin/Aprem

In [18]:
def createSchedule(Df):
    DicoJour = {}
    IsAprem = False
    LstSemaine = []
    # parcour des jours pour séparer matin et aprem
    for obj in Df :
        LstMatin = []
        LstAprem = []
        LstTemp = []
        # parcour des cases non nulles 
        for case in Df[obj] :
            if case != 0 :
                LstTemp.append(case.strip())

        # parcour des cases non nulles d'une journée pour la séparation
        for index,value in enumerate(LstTemp) :
            if index == 0 :
                # on ajoute le jour
                DicoJour["jour"] = obj +"-" + value
            else :
                if value in {'13h-14h30','13h30-17h45','13h10-18h', '13h30-15h30', '15h45-17h45', '13h30 - 14h30','13h30 - 17h45','13h30 - 17h30','13h30-15h45','13h30 - 15h30'} :
                  IsAprem = True
                elif value in {'8h-12h15','8h - 12h15', '8h30 - 10h00', '8h-10h', '10h15-12h15','10h15 - 12h15','8h00 - 10h00', '8h- 10h','8h - 10h','8h00 - 12h15'}:
                    IsAprem = False
                if "\n" in value :
                    # on ajoute les cours
                    if IsAprem :
                        LstAprem.append(value.split("\n")[0])
                        LstAprem.append(value.split("\n")[1])
                    else :
                        LstMatin.append(value.split("\n")[0])
                        LstMatin.append(value.split("\n")[1])
                    continue
                if IsAprem :
                    LstAprem.append(value)
                else :
                    LstMatin.append(value)

        DicoJour["Matin"] = LstMatin.copy()
        DicoJour["Aprem"] = LstAprem.copy()
        LstSemaine.append(DicoJour.copy())
    return LstSemaine

LstSemaine = createSchedule(Df)
df_semaine = pd.DataFrame( LstSemaine)
df_semaine
df_semaine.to_json('planning_semaineMatin_Aprem_TEST.json',orient='records', indent=4)

### On filtre les Majeurs et les GROUPES par jour/ Demi-journée et majeurs ou tronc commun 4ETI

In [1]:
import copy
def filtreMajGrp(pLstSemaine):
    Semaine =  {}
    lstGroupesinit = ['A','B','C','D','TousLesGroupes']
    lstMajeursinit = ['CONCEP.LOGICIELLE/BIG DATA','ROBOTIQUE','ELECTRONIQUE ET SYST EMB','INFRA DES RESEAUX','IMAGE','Pour tous']

    # creation dico {grp: [], ...}
    dicGrpinit = {}
    for grp in lstGroupesinit :
        dicGrpinit[grp] = []
    
    # creation dico {majeur: {grp: [], ...}, ...}
    dicMajinit = {}
    for maj in lstMajeursinit :
        dicMajinit[maj] = copy.deepcopy(dicGrpinit)
    
    print(f'{dicMajinit=}')

    print(f"test: {id(dicMajinit['ROBOTIQUE']['A'])=}", f"and {id(dicMajinit['IMAGE']['A'])=}")

    # parcours des jours de la semaine
    for jour in pLstSemaine :
        dicMaj = copy.deepcopy(dicMajinit)
        print(f'{jour["jour"]=}')
        Majeur = "Pour tous" #on initialise le majeur à tous
        grp = "TousLesGroupes" #on initialise le groupe à tous
        # creation de la journee ("LUNDI-..." : {})
        Semaine[jour["jour"]] = {}

        # parcours de la matinée et on les ajoute au majeur correspondant 
        for Case in jour["Matin"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            elif Case in lstGroupesinit:
                grp = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur][grp].append(Case)
        Semaine[jour["jour"]]["Matin"] = copy.deepcopy(dicMaj)

        # parcours de l'aprem et on les ajoute au majeur correspondant
        dicMaj = copy.deepcopy(dicMajinit)
        for Case in jour["Aprem"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            elif Case in lstGroupesinit:
                grp = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur][grp].append(Case)
        Semaine[jour["jour"]]["Aprem"] = copy.deepcopy(dicMaj)

    print('fin')
    return Semaine

PlanningMajeurGrp = filtreMajGrp(copy.deepcopy(LstSemaine)) # KK
df_planningMajGrp = pd.DataFrame(PlanningMajeurGrp)
df_planningMajGrp

NameError: name 'LstSemaine' is not defined

## On filtres les majeurs de chaque jours

In [5]:
import copy
def filtreMaj(pLstSemaine):
    Semaine =  {}
    lstMajeursinit = ['CONCEP.LOGICIELLE/BIG DATA','ROBOTIQUE','ELECTRONIQUE ET SYST EMB','INFRA DES RESEAUX','IMAGE','Pour tous']
    
    # creation dico {majeur: {grp: [], ...}, ...}
    dicMajinit = {}
    for maj in lstMajeursinit:
        dicMajinit[maj] = []    

    # parcours des jours de la semaine
    for jour in pLstSemaine :
        dicMaj = copy.deepcopy(dicMajinit)
        print(f'{jour["jour"]=}')
        Majeur = "Pour tous" #on initialise le majeur à tous
        Semaine[jour["jour"]] = {}

        # parcours de la matinée et on les ajoute au majeur correspondant 
        for Case in jour["Matin"]:
            # print(f'{Case=}')
            if Case in lstMajeursinit:
                Majeur = Case
            # print(f'{Majeur} - {grp} - {Case}')
            dicMaj[Majeur].append(Case)
        Semaine[jour["jour"]]["Matin"] = copy.deepcopy(dicMaj)

        # parcours de l'aprem et on les ajoute au majeur correspondant
        dicMaj = copy.deepcopy(dicMajinit)
        Majeur = "Pour tous" #on initialise le majeur à tous
        for Case in jour["Aprem"]:
            if Case in lstMajeursinit:
                Majeur = Case
            dicMaj[Majeur].append(Case)
        Semaine[jour["jour"]]["Aprem"] = copy.deepcopy(dicMaj)

    print('fin')
    return Semaine

PlanningMajeur = filtreMaj(copy.deepcopy(LstSemaine)) # KK
df_planningMaj = pd.DataFrame(PlanningMajeur)
df_planningMaj

jour["jour"]='LUNDI-03-avr'
jour["jour"]='MARDI-04-avr'
jour["jour"]='MERCREDI-05-avr'
jour["jour"]='JEUDI-06-avr'
jour["jour"]='VENDREDI-07-avr'
fin


,LUNDI-03-avr,MARDI-04-avr,MERCREDI-05-avr,JEUDI-06-avr,VENDREDI-07-avr
Matin,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...
Aprem,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'..."


### Gestion des 4CGP

In [ ]:
def filtre4CGP(LstSemaine) : # KK
    Semaine =  {}
    for jour in LstSemaine :
        print(jour["jour"])
        Majeur = "Pour tous" #on initialise le majeur à tous
        Mat= {
            "Groupe A":[],
            "Groupe B":[],
            "Groupe C":[],
            "Chimie Médicinale et Hétérocycles":[],
            "Génie de la réaction chimique avancée":[],
            "Transition Energétique":[],
            "Analyse en ligne" : [],
            "Synthèse Macromoléculaire":[],
            "Pour tous":[]
        } 

filtre4CGP(LstSemaine)

In [20]:
with open("3CGPTEST.json", 'w+') as f:
    json.dump(PlanningGroupe, f, indent=4, cls=NumpyEncoder)



# Pour les 3 ETI

In [27]:
import copy
def filtreMaj(pLstSemaine):
    Semaine =  {}
    lstGroupesinit = ['GROUPE A','GROUPE B','GROUPE C','GROUPE D','Pour tous']
    
    # creation dico {majeur: {grp: [], ...}, ...}
    dicGrpinit = {}
    for grp in lstGroupesinit:
        dicGrpinit[grp] = []    

    # parcours des jours de la semaine
    for jour in pLstSemaine :

        print(f'{jour["jour"]=}')
        Semaine[jour["jour"]] = {}
        demi_jour = "Matin", "Aprem"
        # parcours de la matinée et on les ajoute au majeur correspondant 

        for dj in demi_jour:
            isGroupe = False
            count = 0
            dicGrp = copy.deepcopy(dicGrpinit)
            Groupe = "Pour tous"

            for i,Case in enumerate(jour[dj]):
                # print(f'{Case=}')
                if Case in lstGroupesinit:
                    Groupe = Case
                    isGroupe = True
                    dicGrp[Groupe].append(jour[dj][i-1])
                    dicGrp["Pour tous"].pop(-1)
                elif count == 3:
                    isGroupe = False
                    count = 0
                    Groupe = "Pour tous"
                elif isGroupe:
                    count += 1
                # print(f'{Majeur} - {grp} - {Case}')
                dicGrp[Groupe].append(Case)
            Semaine[jour["jour"]][dj] = copy.deepcopy(dicGrp)
    print('fin')
    return Semaine

PlanningGroupe = filtreMaj(copy.deepcopy(LstSemaine)) # KK
df_planningGrp = pd.DataFrame(PlanningGroupe)
df_planningGrp

jour["jour"]='LUNDI-06/02/2023'
jour["jour"]='MARDI-07/02/2023'
jour["jour"]='MERCREDI-08/02/2023'
jour["jour"]='JEUDI-09/02/2023'
jour["jour"]='VENDREDI-10/02/2023'
fin


,LUNDI-06/02/2023,MARDI-07/02/2023,MERCREDI-08/02/2023,JEUDI-09/02/2023,VENDREDI-10/02/2023
Matin,"{'GROUPE A': [], 'GROUPE B': [], 'GROUPE C': [...","{'GROUPE A': ['8h-10h', 'GROUPE A', 'TD SP2', ...","{'GROUPE A': [], 'GROUPE B': [], 'GROUPE C': [...","{'GROUPE A': [], 'GROUPE B': [], 'GROUPE C': [...","{'GROUPE A': [], 'GROUPE B': [], 'GROUPE C': [..."
Aprem,"{'GROUPE A': ['13h30-15h45', 'GROUPE A', 'TD P...","{'GROUPE A': [], 'GROUPE B': [], 'GROUPE C': [...","{'GROUPE A': [], 'GROUPE B': [], 'GROUPE C': [...","{'GROUPE A': [], 'GROUPE B': [], 'GROUPE C': [...","{'GROUPE A': [], 'GROUPE B': [], 'GROUPE C': [..."


# Pour les 3CGP

In [19]:
import copy
def filtreMaj(pLstSemaine):
    Semaine =  {}
    lstGroupesinit = ['Groupe 1','Groupe 2','Groupe 3','Pour tous']
    
    # creation dico {majeur: {grp: [], ...}, ...}
    dicGrpinit = {}
    for grp in lstGroupesinit:
        dicGrpinit[grp] = []    

    # parcours des jours de la semaine
    for jour in pLstSemaine :

        print(f'{jour["jour"]=}')
        Semaine[jour["jour"]] = {}
        demi_jour = ("Matin", "Aprem")
        # parcours de la matinée et on les ajoute au majeur correspondant 

        for dj in demi_jour:
            dicGrp = copy.deepcopy(dicGrpinit)
            Groupe = "Pour tous"

            for Case in jour[dj]:
                if Case in lstGroupesinit:
                    Groupe = Case
                dicGrp[Groupe].append(Case)
            Semaine[jour["jour"]][dj] = copy.deepcopy(dicGrp)
    print('fin')
    return Semaine

PlanningGroupe = filtreMaj(copy.deepcopy(LstSemaine)) # KK
df_planningGrp = pd.DataFrame(PlanningGroupe)
df_planningGrp

jour["jour"]='LUNDI-03-avr'
jour["jour"]='MARDI -04-avr'
jour["jour"]='MERCREDI-05-avr'
jour["jour"]='JEUDI-06-avr'
jour["jour"]='VENDREDI-07-avr'
fin


,LUNDI-03-avr,MARDI -04-avr,MERCREDI-05-avr,JEUDI-06-avr,VENDREDI-07-avr
Matin,"{'Groupe 1': [], 'Groupe 2': [], 'Groupe 3': [...","{'Groupe 1': [], 'Groupe 2': [], 'Groupe 3': [...","{'Groupe 1': [], 'Groupe 2': [], 'Groupe 3': [...","{'Groupe 1': ['Groupe 1', 'Mathématiques Infor...","{'Groupe 1': [], 'Groupe 2': [], 'Groupe 3': [..."
Aprem,"{'Groupe 1': ['Groupe 1', 'Sciences Analytique...","{'Groupe 1': [], 'Groupe 2': [], 'Groupe 3': [...","{'Groupe 1': [], 'Groupe 2': [], 'Groupe 3': [...","{'Groupe 1': [], 'Groupe 2': [], 'Groupe 3': [...","{'Groupe 1': ['Groupe 1', 'Génie des Procédés'..."
